In [1]:
from msa_sp_score_3k import *
from alignment import *
from parsing import parse_matrix_and_gap_cost_in_subst_matrix,read_n_fasta
import os

In [2]:
def print_usage():
	print("""
Usage: msa_sp_score.py <filename>

where <filename> contains a multiple sequence alignment over the alphabet
{a,c,g,t,-} in FASTA format.
""")

##########################################################################
# Cost parameters and helper functions
##########################################################################

cost = [[0, 5, 2, 5, 5],  # A
        [5, 0, 5, 2, 5],  # C
        [2, 5, 0, 5, 5],  # G
        [5, 2, 5, 0, 5],  # T
        [5, 5, 5, 5, 0]]  #-'

dict_str2seq = {'a':0, 'c':1, 'g':2, 't':3, 'A':0, 'C':1, 'G':2, 'T':3, '-':4, 'N':0, 'R':0, 'S':0}

def str2seq(s):
    try:
        seq = [dict_str2seq[c] for c in list(s)]
        return seq
    except KeyError as e:
        print("ERROR: Illegal character", e, "in input string.")
        sys.exit(1)

###########################################################################
# read_fasta(filename)
#
# Returns an array of strings. The strings correspond to the strings
# defined in the FASTA file 'filename' with whitespaces removed.
###########################################################################

def read_fasta(filename):
	f = open(filename)
	res = []
	curr_str = ""
	curr_name = ""
	for l in f.readlines():
		l = l.strip()
		# Ignore empty lines
		if len(l) == 0:
			continue
		# Ignore comment lines
		elif l[0] == ';':
			continue
		# A new string starts when a new name is found
		elif l[0] == '>':
			if curr_name != "":
				res.append(remove_whitespaces(curr_str))
			curr_str = ""
			curr_name = l[1:].strip()
		# Otherwise append current line to the current string
		else:
			curr_str = curr_str + l
	res.append(remove_whitespaces(curr_str))
	return res

def remove_whitespaces(s):
	return "".join(s.split())

def compute_sp_score_cust(filename,cost = cost):	
	# Read FASTA file and convert input strings to sequences
	row = []
	for s in read_fasta(filename):
		row.append(str2seq(s))
	# Compute the score of each induced pairwise alignment
	score = 0
	for i in range(len(row)):
		for j in range(i+1, len(row)):
			if len(row[i]) != len(row[j]):
				print("ERROR: Rows", i, "and", j, "have different lengths.")
				sys.exit(1)
			for c in range(len(row[i])):
				score = score + cost[row[i][c]][row[j][c]]
	return score

In [3]:
substitution_matrix ={'A': {'A': 0, 'C': 5, 'G': 5, 'T': 5, '*': 5}, 
                    'C': {'A': 5, 'C': 0, 'G': 5, 'T': 2, '*': 5}, 
                    'G': {'A': 2, 'C': 5, 'G': 0, 'T': 5, '*': 5}, 
                    'T': {'A': 5, 'C': 2, 'G': 5, 'T': 0, '*': 5},
                    '-': {'A': 5, 'C': 5, 'G': 5, 'T': 5, '*': 5},
                    '*': {'A': 5, 'C': 5, 'G': 5, 'T': 5, '*': 5}}

def backtrack(seq1,seq2, gapcost,M):
    alignment1, alignment2 = "", ""
    i, j = len(seq1), len(seq2)
    while i > 0 or j > 0:
        if i > 0 and M[i][j] == M[i-1][j] + gapcost:
            alignment1 = seq1[i-1] + alignment1
            alignment2 = "-" + alignment2
            i -= 1
        elif j > 0 and M[i][j] == M[i][j-1] + gapcost:
            alignment1 = "-" + alignment1
            alignment2 = seq2[j-1] + alignment2
            j -= 1
        else:
            alignment1 = seq1[i-1] + alignment1
            alignment2 = seq2[j-1] + alignment2
            i -= 1
            j -= 1
    return alignment1, alignment2


def pariwise_alignment(seq1, seq2,subst_matrix,gap_penalty):
    # Initialize the matrix with zeros
    M = [[0] * (len(seq2) + 1) for i in range(len(seq1) + 1)]
    # Initialize the first row and column
    for i in range(1, len(seq1) + 1):
        M[i][0] = M[i-1][0] + gap_penalty
    for j in range(1, len(seq2) + 1):
        M[0][j] = M[0][j-1] + gap_penalty
    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            match_score = M[i-1][j-1] + subst_matrix[seq1[i-1]][seq2[j-1]]
            delete_score = M[i-1][j] + gap_penalty
            insert_score = M[i][j-1] + gap_penalty
            M[i][j] = min(match_score, delete_score, insert_score)
    return M

def reference_alignment(seq1, seq2):
    ref_alignment = ''
    for a, b in zip(seq1, seq2):
        if a == '-' and b != '-':
            ref_alignment += b
        elif a != '-' and b == '-':
            ref_alignment += a
        elif a == b:
            ref_alignment += a
        else:
            ref_alignment += '*'
    # Handle any remaining nucleotides in the longer sequence
    if len(seq1) > len(seq2):
        ref_alignment += seq1[len(seq2):].replace('-', '')
    elif len(seq2) > len(seq1):
        ref_alignment += seq2[len(seq1):].replace('-', '')
    
    return ref_alignment

def add_row(matrix, new_row):
    # Determine the length of the new row
    new_row_length = len(new_row)
    # Determine the length of the existing rows
    existing_row_lengths = [len(row) for row in matrix]
    max_row_length = max(existing_row_lengths)
    # If the new row is shorter than the longest existing row, add dashes to the end
    if new_row_length < max_row_length:
        new_row += '-' * (max_row_length - new_row_length)
    # If the new row is longer than the longest existing row, add dashes to the end of all existing rows
    if new_row_length > max_row_length:
        for i in range(len(matrix)):
            matrix[i] += '-' * (new_row_length - max_row_length)
    matrix.append(new_row)
    return matrix

def capibara_slow_alignement(list_of_seqs,substitution_matrix,gapcost):
    #initialization
    M = pariwise_alignment(list_of_seqs[0],list_of_seqs[1],substitution_matrix,gapcost)
    v_aligned, w_aligned = backtrack(list_of_seqs[0], list_of_seqs[1], gapcost, M)
    ref_alignment = reference_alignment(v_aligned, w_aligned)
    matrix = [[i for i in v_aligned],[i for i in w_aligned]]
    #iterate over the list of sequences from 2 to the end
    for i in range(2,len(list_of_seqs)):
        # compute the pairwise alignment matrix between the reference alignment and the current sequence
        print("Aligning pairwise: ",i)
        temp = pariwise_alignment(list_of_seqs[i],ref_alignment,substitution_matrix,gapcost)
        # compute the alignment between the current sequence and the reference alignment
        v_aligned, w_aligned = backtrack(list_of_seqs[i], ref_alignment, gapcost, temp)
        # update the reference alignment
        ref_alignment = reference_alignment(v_aligned, w_aligned)
        # update the matrix
        new_row = [i for i in v_aligned]
        add_row(matrix,new_row)
    return matrix



Read and align the sequences from 1-3 to 1-6.

In [4]:
matrix_path = 'input/subst_matrix.txt'
substitution_matrix_app = parse_matrix_and_gap_cost_in_subst_matrix(matrix_path)

In [5]:
%%time
list_of_seq = read_n_fasta('experiments/brca1-testseqs.fasta',6)
results = []
for i in [3,4,5,6]:
        M = two_approx_algorithm_for_MSA(list_of_seq[:i], substitution_matrix_app)  ########## approx algorithm
        alignment_approx_outputfile = 'presentation/approx/aligned_' + str(i)
        approx_alignment = print_alignment(M,alignment_approx_outputfile) ## print alignment to file
    

CPU times: total: 3.2 s
Wall time: 3.46 s


Compute scores from alignments:

In [6]:
for i in [3,4,5,6]:
    ####### approx
    testfile = 'presentation/approx/aligned_' + str(i)
    if os.path.isfile(testfile):   
        score = compute_sp_score(testfile)
        print("The optimal score (approx) is in seqs 1 -",i,":",score)

The optimal score (approx) is in seqs 1 - 3 : 792
The optimal score (approx) is in seqs 1 - 4 : 2354
The optimal score (approx) is in seqs 1 - 5 : 3315
The optimal score (approx) is in seqs 1 - 6 : 5998


Now let's try with our algorithm:

In [7]:
list_of_seq = read_n_fasta('experiments/brca1-full.fasta')

In [8]:
list_of_seq

['...']

In [9]:
%%time
for i in [3,4,5,6]:
        print(len(list_of_seq[:i]))
        M = capibara_slow_alignement(list_of_seq[:i],substitution_matrix,5)
        alignment_approx_outputfile = 'presentation/capi/aligned_' + str(i)
        aligned_seq = [''.join(i) for i in M] 
        save_sequences_as_fasta(alignment_approx_outputfile,aligned_seq,'')
        #approx_alignment = print_alignment(M,alignment_approx_outputfile) ## print alignment to file


3
Aligning pairwise:  2
4
Aligning pairwise:  2
Aligning pairwise:  3
5
Aligning pairwise:  2
Aligning pairwise:  3
Aligning pairwise:  4
6
Aligning pairwise:  2
Aligning pairwise:  3
Aligning pairwise:  4
Aligning pairwise:  5
CPU times: total: 7min 53s
Wall time: 8min 24s


In [10]:
for i in [3,4,5,6]:
    ####### approx
    testfile = 'presentation/capi/aligned_' + str(i)
    if os.path.isfile(testfile):   
        score = compute_sp_score_cust(testfile)
        print("The optimal score (capi) is in seqs 1 -",i,":",score)

The optimal score (capi) is in seqs 1 - 3 : 42558
The optimal score (capi) is in seqs 1 - 4 : 124143
The optimal score (capi) is in seqs 1 - 5 : 215833
The optimal score (capi) is in seqs 1 - 6 : 345940


In [14]:
seq_full = read_n_fasta('experiments/brca1-full.fasta')
len(seq_full)

8

In a sequence, there are nucleotides that are not A,C,G or T

In [15]:
for seq in seq_full:
    print(set(seq))

{'T', 'G', 'A', 'C'}
{'T', 'G', 'A', 'C'}
{'G', 'T', 'A', 'C'}
{'G', 'T', 'A', 'C'}
{'T', 'G', 'A', 'C'}
{'G', 'T', 'A', 'C'}
{'T', 'G', 'A', 'C'}
{'G', 'N', 'T', 'A', 'S', 'R', 'C'}


In [16]:
import random
# Replace 'N', 'R' or 'S' with a random nucleotide
clean_seq_full = []
for sequence in seq_full:
    replaced_sequence = ''
    for nucleotide in sequence:
        if nucleotide == 'N' or nucleotide == 'R' or nucleotide == 'S' :
            replaced_sequence += random.choice(['A', 'C', 'T', 'G'])
        else:
            replaced_sequence += nucleotide
    clean_seq_full.append(replaced_sequence)

for seq in clean_seq_full:
    print(set(seq))

{'T', 'G', 'A', 'C'}
{'T', 'G', 'A', 'C'}
{'G', 'T', 'A', 'C'}
{'G', 'T', 'A', 'C'}
{'T', 'G', 'A', 'C'}
{'G', 'T', 'A', 'C'}
{'T', 'G', 'A', 'C'}
{'G', 'T', 'A', 'C'}


Align the 8 sequences:

In [17]:
test_list = [item[:1000] for item in clean_seq_full]
test_list

['ATGGATTTATCTGCGGATCATGTTGAAGAAGTACAAAATGTCCTCAATGCTATGCAGAAAATCTTAGAGTGTCCAATATGTCTGGAGTTGATCAAAGAGCCTGTCTCTACAAAGTGTGACCACATATTTTGCAAATTTTGTATGCTGAAACTTCTCAACCAGAAGAAAGGGCCTTCACAATGTCCTTTGTGTAAGAATGATATAACCAAAAGAAGCCTACAAGAAAGTACAAGATTTAGTCAACTTGTTGAAGAGCTGTTGAAGATCATTCATGCTTTTGAGCTTGACACAGGATTGCAGTTTGCAAACAGCTATAACTTTTCGAGAAAGGAAGATAACTCTCCTGAGCATCTGAAGGAGGAAGTTTCTATAATCCAAAGTATGGGCTACCGGAACCGGGCCAAAAGACTTTGGCAGAGTGAACCTGAAAATCCTACCTTGCAGGAAACCAGTCTTACTGTCGAACTCTCTAACCTTGGAATTGTCAGATCTCTGAGGACAAAGCAGCAGACACAATCTCAGAATAAGTCCGTCTACATTGAATTGGGATCTGATTCTTCTGAAGACACAGTTAATAAGGCCAGTTATTTCAGTGTGGGAGACCATGAATTGTTAGAAATCACCCCTCAAGGAGCCAAGGCCAAGACCAATTTGAATCCTGCAGAAAAGGCTGCTTGTGAGTTTTCTGAGAAGGACATAACAAATACTGAACATCATCAACTCAGTATTAAAGATCTGATCACCACCCAGAAGCATGCAACTGAGACGCATCCAGAAAAGTATCAGGGTATTTCTGTTTCAGACTTTCACGTGGAGCCATGTGGCACAGATACTCATGCCAGCTCATTACAGCATGAGAACAGCAGTTTATTGCTCACTGAAAACAGACTGAATGTAGAAAAGGCTGAATTCTGTAATAAAAGCAAACAGCCTGTCTTAGTAAAGAGCCAGCAGAGCAGATGGGCTGAAAGTAAGGGCACATGTAAGGATAGGCAGAT

In [19]:
%%time
M = two_approx_algorithm_for_MSA(clean_seq_full, substitution_matrix_app)
alignment = print_alignment(M)
alignment_outputfile = 'presentation/full_aligned_approx' 
save_sequences_as_fasta(alignment_outputfile,aligned_seq,'')
score = compute_sp_score(alignment_outputfile)
print("The score with the approx: ",score)

In [29]:
%%time
M = capibara_slow_alignement(clean_seq_full,substitution_matrix,5)
alignment_outputfile = 'presentation/full_aligned' 
aligned_seq = [''.join(i) for i in M] 
save_sequences_as_fasta(alignment_outputfile,aligned_seq,'')

Aligning pairwise:  2
Aligning pairwise:  3
Aligning pairwise:  4
Aligning pairwise:  5
Aligning pairwise:  6
Aligning pairwise:  7
CPU times: total: 4min 7s
Wall time: 4min 38s


In [30]:
compute_sp_score(alignment_outputfile)

666566